#### Move the translational stage and record the positions

In [1]:
# Generate control code for stage movement
import numpy as np
from pycromanager import Core, Acquisition
import random
import time
import math

# Connect to the Micro-Manager Core to get initial state
core = Core()
print(core)

xy_stage = core.get_xy_stage_device()
print(f"Using XY Stage: {xy_stage}")

if xy_stage:
    start_x = core.get_x_position(xy_stage)
    start_y = core.get_y_position(xy_stage)
    print(f"Initial Position: X={start_x}, Y={start_y}")
else:
    # Fallback for testing if no stage present
    start_x, start_y = 0, 0
    print("No XY Stage found. Using (0,0).")

Using XY Stage: XYStage
Initial Position: X=13798.4, Y=-1614.2


In [2]:
stage_coordinates = []
n_ROI = 5
for i in range(n_ROI):
    input(f"Please move stage manually to ROI {i+1}, then press enter to confirm:")
    coordinate_temp_x = core.get_x_position(xy_stage)
    coordinate_temp_y = core.get_y_position(xy_stage)
    coordinate_temp = [coordinate_temp_x, coordinate_temp_y]
    stage_coordinates.append(coordinate_temp)

stage_coordinates = np.array(stage_coordinates)
print(stage_coordinates[:3])

[[13798.4 -1614.2]
 [14174.1 -1614.2]
 [13930.5 -2386.5]]


##### Move stage and record data

In [3]:
auto_mode = True
type_str = 'apoptosis' # 'necrosis'

if auto_mode:
     # Define a directory to save data (optional)
    save_dir = r"D:\pengchen2026\repositories\WF-Acq-py-main\AcquisitionData"
    acq_name = "CellDeathEventsAcquisition"
    events_list = []
    for i in range(n_ROI):
        # Note: xy_stage must be defined (it is from cell 2)
        target_x, target_y = stage_coordinates[i]
        # Generate acquisition events
        evt = {
                'axes': {'position': i, 'type': type_str},
                'x': target_x, # positions in um
                'y': target_y, # positions in um
                'exposure': 100, # in ms
            }
        events_list.append(evt)
        
    # Show first few events
    print(events_list[:3])

    # Run the Acquisition
    print("Starting Acquisition...")
    with Acquisition(directory=save_dir, name=acq_name) as acq:
        acq.acquire(events_list)

else:
    for i in range(n_ROI):
        # Note: xy_stage must be defined (it is from cell 2)
        temp_x, temp_y = stage_coordinates[i]
        core.set_xy_position(xy_stage, temp_x, temp_y)
        core.wait_for_device(xy_stage)
        input(f"Stage has been moved to ROI {i+1}, plase capture the frame, then press enter to proceed:")

print("Acquisition complete.")

[{'axes': {'position': 0, 'type': 'apoptosis'}, 'x': 13798.4, 'y': -1614.2, 'exposure': 100}, {'axes': {'position': 1, 'type': 'apoptosis'}, 'x': 14174.1, 'y': -1614.2, 'exposure': 100}, {'axes': {'position': 2, 'type': 'apoptosis'}, 'x': 13930.5, 'y': -2386.5, 'exposure': 100}]
Starting Acquisition...
Acquisition complete.
